In [1]:
import sagemaker
#sagemaker.local.LocalSession
#Use local session for traning locally other wise use session
#sess=sagemaker.local.LocalSession()
sess=sagemaker.Session()
role= sagemaker.get_execution_role()
account =sess.boto_session.client('sts').get_caller_identity()['Account']
region= sess.boto_session.region_name

In [2]:
region

'us-east-1'

In [3]:
!cat Dockerfile

#FROM nvidia/cuda:11.2.1-cudnn8-runtime-ubuntu20.04
FROM nvidia/cuda:11.0-runtime
ENV DEBIAN_FRONTEND=noninteractive 
RUN apt-get update && apt-get install -y --no-install-recommends \
		ca-certificates \
		curl \
		netbase \
		wget \
       build-essential \
        libopencv-dev\
        libopenblas-dev \
        nginx

ENV PYTHON_VERSION=3.6

ENV LANG=C.UTF-8 LC_ALL=C.UTF-8
ENV PATH /opt/conda/bin:$PATH

RUN wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-4.5.12-Linux-x86_64.sh -O ~/miniconda.sh && \
    /bin/bash ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda install -y python=$PYTHON_VERSION 

RUN pip install tensorflow-gpu

#COPY ./tensorflow-2.3.0-cp38-cp38-linux_x86_64.whl /opt/
#RUN pip install /opt/tensorflow-2.3.0-cp38-cp38-linux_x86_64.whl 
RUN pip install scipy pillow gunicorn==19.9.0 gevent flask

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"

COPY ./scripts /opt/program

RU

In [4]:
repo_name="k-fruit-class"
image_tag="k-fruits-class-gpu"
base_job_name="k-fruits-cnn"

%env dockerfile Dockerfile

train_instance_type ="ml.p3.8xlarge"

env: dockerfile=Dockerfile


### Below cells sets up enironment variables build dokcer image and push to ECR If the image is already in ECR skip these steps

In [5]:
%env account {account}
%env region {region}
%env repo_name {repo_name}
%env image_tag {image_tag}

env: account=470086202700
env: region=us-east-1
env: repo_name=k-fruit-class
env: image_tag=k-fruits-class-gpu


In [6]:
%%sh

aws ecr describe-repositories --repository-names $repo_name > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name $repo_name > /dev/null
fi

$(aws ecr get-login --region $region --no-include-email)

Login Succeeded


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [9]:
!docker build -t $image_tag -f $dockerfile .

Sending build context to Docker daemon  1.099GB
Step 1/16 : FROM nvidia/cuda:11.0-cudnn8-runtime-ubuntu18.04
11.0-cudnn8-runtime-ubuntu18.04: Pulling from nvidia/cuda

57c49d0f: Pulling fs layer 
40447d26: Pulling fs layer 
2f862619: Pulling fs layer 
278deddf: Pulling fs layer 
80049843: Pulling fs layer 
556b2329: Pulling fs layer 
a0c97a55: Pulling fs layer 
Digest: sha256:f436a0fcd117ba20d3609da55417f1e8880fdb88a21e05afbccd3243f8895c60Downloading  902.1MB/1.134GBExtracting  223.9MB/1.134GB
Status: Downloaded newer image for nvidia/cuda:11.0-cudnn8-runtime-ubuntu18.04
 ---> 848be2582b0a
Step 2/16 : ENV DEBIAN_FRONTEND=noninteractive
 ---> Running in c511df02d3db
Removing intermediate container c511df02d3db
 ---> b87eb9d25fab
Step 3/16 : RUN apt-get update && apt-get install -y --no-install-recommends 		ca-certificates 		curl 		netbase 		wget        build-essential         libopencv-dev        libopenblas-dev         nginx
 ---> Running in 73ebe03ab968
Ign:1 https://developer.downloa

The following NEW packages will be installed:
  adwaita-icon-theme binutils binutils-common binutils-x86-64-linux-gnu
  build-essential cpp cpp-7 curl dconf-gsettings-backend dconf-service
  dpkg-dev fontconfig fontconfig-config fonts-dejavu-core g++ g++-7 gcc gcc-7
  gcc-7-base gdal-data glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas gtk-update-icon-cache
  hicolor-icon-theme humanity-icon-theme iproute2 libaec0 libarmadillo8
  libarpack2 libasan4 libatk-bridge2.0-0 libatk1.0-0 libatk1.0-data libatomic1
  libatspi2.0-0 libavahi-client3 libavahi-common-data libavahi-common3
  libavcodec-dev libavcodec57 libavformat-dev libavformat57 libavresample-dev
  libavresample3 libavutil-dev libavutil55 libbinutils libbluray2 libbsd0
  libc-dev-bin libc6-dev libcairo-gobject2 libcairo2 libcc1-0 libcharls1
  libchromaprint1 libcilkrts5 libcolord2 libcroco3 libcrystalhd3 libcups2
  libcurl3-gnutls libcurl4 libdap25 libdapclient6v5 libdatrie1 libdbus-1-3


Get:30 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgdbm5 amd64 1.14.1-6 [26.0 kB]
Get:31 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgdbm-compat4 amd64 1.14.1-6 [6084 B]
Get:32 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libperl5.26 amd64 5.26.1-6ubuntu0.5 [3534 kB]
Get:33 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 perl amd64 5.26.1-6ubuntu0.5 [201 kB]
Get:34 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 ca-certificates all 20210119~18.04.1 [147 kB]
Get:35 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libelf1 amd64 0.170-0.4ubuntu0.1 [44.8 kB]
Get:36 http://archive.ubuntu.com/ubuntu bionic/main amd64 libmnl0 amd64 1.0.4-2 [12.3 kB]
Get:37 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 iproute2 amd64 4.15.0-2ubuntu1.2 [722 kB]
Get:38 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libdbus-1-3 amd64 1.12.2-1ubuntu1.2 [175 kB]
Get:39 http://archive.ubuntu.com/ubuntu bionic-updates/main amd6

Get:111 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libgcc-7-dev amd64 7.5.0-3ubuntu1~18.04 [2378 kB]
Get:112 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 gcc-7 amd64 7.5.0-3ubuntu1~18.04 [9381 kB]
Get:113 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 gcc amd64 4:7.4.0-1ubuntu2.3 [5184 B]
Get:114 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libstdc++-7-dev amd64 7.5.0-3ubuntu1~18.04 [1471 kB]
Get:115 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 g++-7 amd64 7.5.0-3ubuntu1~18.04 [9697 kB]
Get:116 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 g++ amd64 4:7.4.0-1ubuntu2.3 [1568 B]
Get:117 http://archive.ubuntu.com/ubuntu bionic/main amd64 make amd64 4.1-9.1ubuntu1 [154 kB]
Get:118 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libdpkg-perl all 1.19.0.5ubuntu2.3 [211 kB]
Get:119 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 patch amd64 2.7.6-2ubuntu1.1 [102 kB]
Get:120 http://archive.

Get:189 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libavformat-dev amd64 7:3.4.8-0ubuntu0.2 [1132 kB]
Get:190 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libavresample3 amd64 7:3.4.8-0ubuntu0.2 [52.6 kB]
Get:191 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libavresample-dev amd64 7:3.4.8-0ubuntu0.2 [62.0 kB]
Get:192 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libcairo-gobject2 amd64 1.15.10-2ubuntu0.1 [17.1 kB]
Get:193 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libcharls1 amd64 1.1.0+dfsg-2 [56.1 kB]
Get:194 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 liblcms2-2 amd64 2.9-1ubuntu0.1 [139 kB]
Get:195 http://archive.ubuntu.com/ubuntu bionic/main amd64 libcolord2 amd64 1.3.3-2build1 [107 kB]
Get:196 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libcups2 amd64 2.2.7-1ubuntu2.8 [211 kB]
Get:197 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libcurl3-gnutls amd64 7.58.0

...

Setting up libcharls1:amd64 (1.1.0+dfsg-2) ...
Setting up libtbb2:amd64 (2017~U7-8) ...
Setting up libelf1:amd64 (0.170-0.4ubuntu0.1) ...
Setting up libtsan0:amd64 (8.4.0-1ubuntu1~18.04) ...
Setting up libgeos-3.6.2:amd64 (3.6.2-1build2) ...
Setting up libsoxr0:amd64 (0.1.2-3) ...
Setting up libglib2.0-0:amd64 (2.56.4-0ubuntu0.18.04.6) ...
Setting up nginx-common (1.14.0-0ubuntu1.7) ...
Setting up mysql-common (5.8+1.0.4) ...
update-alternatives: using /etc/mysql/my.cnf.fallback to provide /etc/mysql/my.cnf (my.cnf) in auto mode
Setting up libxshmfence1:amd64 (1.3-1) ...
Setting up libopencv-core3.2:amd64 (3.2.0+dfsg-4ubuntu0.1) ...
Setting up libwayland-client0:amd64 (1.16.0-1ubuntu1.1~18.04.3) ...
Setting up xkb-data (2.23.1-1ubuntu1.18.04.1) ...
Setting up libproxy1v5:amd64 (0.4.15-1ubuntu0.2) ...
Setting up linux-libc-dev:amd64 (4.15.0-135.139) ...
Setting up libmpfr6:amd64 (4.0.1-1) ...
Setting up libsocket++1:amd64 (1.12.13-9) ...
Setting up libgdk-pixbuf2.0-common (2.36.11-2) ..

Setting up netbase (5.4) ...
Setting up libwebp6:amd64 (0.6.1-2) ...
Setting up zlib1g-dev:amd64 (1:1.2.11.dfsg-0ubuntu2) ...
Setting up libjpeg8:amd64 (8c-2ubuntu8) ...
Setting up libisl19:amd64 (0.19-1) ...
Setting up libjsoncpp1:amd64 (1.7.4-3) ...
Setting up libvorbisfile3:amd64 (1.3.5-4.2) ...
Setting up libraw1394-dev:amd64 (2.1.2-1) ...
Setting up libk5crypto3:amd64 (1.16-2ubuntu0.2) ...
Setting up libopencv-photo3.2:amd64 (3.2.0+dfsg-4ubuntu0.1) ...
Setting up libgphoto2-port12:amd64 (2.5.16-2) ...
Setting up libdap25:amd64 (3.19.1-2build1) ...
Setting up libproj12:amd64 (4.9.3-2) ...
Setting up fontconfig-config (2.12.6-0ubuntu2) ...
Setting up libodbc1:amd64 (2.3.4-1.1ubuntu3) ...
Setting up libnginx-mod-stream (1.14.0-0ubuntu1.7) ...
Setting up libasan4:amd64 (7.5.0-3ubuntu1~18.04) ...
Setting up libgfortran4:amd64 (7.5.0-3ubuntu1~18.04) ...
Setting up libilmbase-dev (2.2.0-11ubuntu2) ...
Setting up dconf-service (0.26.0-2ubuntu3) ...
Setting up libbinutils:amd64 (2.30-21ubu

Setting up libxxf86vm1:amd64 (1:1.1.4-1) ...
Setting up libva-x11-2:amd64 (2.1.0-3) ...
Setting up libxrandr2:amd64 (2:1.5.1-1) ...
Setting up libxerces-c3.2:amd64 (3.2.0+debian-2) ...
Setting up libglx-mesa0:amd64 (20.0.8-0ubuntu1~18.04.1) ...
Setting up libxi6:amd64 (2:1.7.9-1) ...
Setting up libcairo2:amd64 (1.15.10-2ubuntu0.1) ...
Setting up libxinerama1:amd64 (2:1.1.3-1) ...
Setting up libnetcdf-c++4 (4.2-8) ...
Setting up libhdf5-openmpi-100:amd64 (1.10.0-patch1+docs-4) ...
Setting up libavutil55:amd64 (7:3.4.8-0ubuntu0.2) ...
Setting up curl (7.58.0-2ubuntu3.12) ...
Setting up build-essential (12.4ubuntu1) ...
Setting up libtk8.6:amd64 (8.6.8-4) ...
Setting up libnginx-mod-http-image-filter (1.14.0-0ubuntu1.7) ...
Setting up libcairo-gobject2:amd64 (1.15.10-2ubuntu0.1) ...
Setting up libgphoto2-6:amd64 (2.5.16-2) ...
Setting up libswresample2:amd64 (7:3.4.8-0ubuntu0.2) ...
Setting up libpangoft2-1.0-0:amd64 (1.40.14-1ubuntu0.1) ...
Setting up pkg-config (0.29.1-0ubuntu2) ...
Set

sqlite-3.33.0        | 2.0 MB    | ########## | 100% 
ca-certificates-2021 | 128 KB    | ########## | 100% 
## Package Plan ##

  environment location: /opt/conda

  added / updated specs: 
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1j             |       h27cfd23_0         3.8 MB
    ncurses-6.2                |       he6710b0_1         1.1 MB
    ld_impl_linux-64-2.33.1    |       h53a641e_7         645 KB
    libffi-3.3                 |       he6710b0_2          54 KB
    _libgcc_mutex-0.1          |             main           3 KB
    pip-21.0.1                 |   py36h06a4308_0         2.0 MB
    certifi-2020.12.5          |   py36h06a4308_0         144 KB
    python-3.6.12              |       hcff3b4d_2        34.0 MB
    tk-8.6.10                  |       hbc83047_0         3.2 MB
    libgcc-ng-9.1.0            |       hdf63c60_0         8.1 

Removing intermediate container fda0a24c3c5a
 ---> 9dd6a1e4336a
Step 9/16 : RUN pip install scipy pillow gunicorn==19.9.0 gevent flask
 ---> Running in 03276854cb84
Removing intermediate container 03276854cb84
 ---> 00af00854a81
Step 10/16 : ENV PYTHONUNBUFFERED=TRUE
 ---> Running in 35f4b4efdcf4
Removing intermediate container 35f4b4efdcf4
 ---> 6416bbf3eb4b
Step 11/16 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Running in c154cbcc281a
Removing intermediate container c154cbcc281a
 ---> b5750e56d242
Step 12/16 : ENV PATH="/opt/program:${PATH}"
 ---> Running in 00d816eff2bc
Removing intermediate container 00d816eff2bc
 ---> 3b47ec8675f7
Step 13/16 : COPY ./scripts /opt/program
 ---> 0ad0521931f3
Step 14/16 : RUN chmod +x /opt/program/train
 ---> Running in adf897ccf0ea
Removing intermediate container adf897ccf0ea
 ---> db1d7004b7e2
Step 15/16 : RUN chmod +x /opt/program/serve
 ---> Running in d1c497738502
Removing intermediate container d1c497738502
 ---> f048f6499823
Step 16/16 : WORKDIR 

In [10]:
!docker tag $image_tag $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

In [11]:
 !docker images

REPOSITORY                                                   TAG                               IMAGE ID            CREATED             SIZE
470086202700.dkr.ecr.us-east-1.amazonaws.com/k-fruit-class   latest                            466e9b7cfcfc        11 minutes ago      6.85GB
k-fruits-class-gpu                                           latest                            466e9b7cfcfc        11 minutes ago      6.85GB
<none>                                                       <none>                            a5627ad31a64        28 minutes ago      5.34GB
nvidia/cuda                                                  11.0-cudnn8-runtime-ubuntu18.04   848be2582b0a        4 months ago        3.6GB
nvidia/cuda                                                  11.0-runtime                      6db6a6e25752        6 months ago        1.95GB


In [12]:
 !docker push $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

The push refers to repository [470086202700.dkr.ecr.us-east-1.amazonaws.com/k-fruit-class]

5be5e65f: Preparing 
d688ed46: Preparing 
e8a84169: Preparing 
acb607eb: Preparing 
4f754065: Preparing 
4edd6bb5: Preparing 
ae4e424d: Preparing 
2ffdcad9: Preparing 
a7c9e3d1: Preparing 
4dce1444: Preparing 
30bcc944: Preparing 
e116c0c0: Preparing 
4df0ad6c: Preparing 
df553184: Preparing 


a7c9e3d1: Pushing  322.5MB/1.825GB

2ffdcad9: Pushing  1.429GB/1.661GB

a7c9e3d1: Pushed   1.825GB/1.825GBlatest: digest: sha256:50b56b27395226eed12a6b8450721fc4820aeccc39a630a52f00da48b2937513 size: 3477


### Upload data to S3 bucket from local repo if data is already in s3 just provide the paths of s3 

In [13]:
local_directory = 'data'
prefix          = repo_name+'/input'

#train_input_path      = sess.upload_data(local_directory+'/training/',      key_prefix=prefix+'/train')
#validation_input_path = sess.upload_data(local_directory+'/validation/', key_prefix=prefix+'/validation')

In [14]:
train_input_path="s3://sagemaker-us-east-1-470086202700/k-fruit-class/input/train"
validation_input_path="s3://sagemaker-us-east-1-470086202700/k-fruit-class/input/validation"

### Run Below cell if the image is already built and you know the name otherwise uncomment the commented part

In [15]:
#output_path = 's3://{}/{}/output'.format(sess.default_bucket(), repo_name)
#image_name  = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, repo_name)
output_path="s3://sagemaker-us-east-1-470086202700/k-fruit-class/output"
image_name="470086202700.dkr.ecr.us-east-1.amazonaws.com/k-fruit-class:latest"
print(output_path)
print(image_name)

s3://sagemaker-us-east-1-470086202700/k-fruit-class/output
470086202700.dkr.ecr.us-east-1.amazonaws.com/k-fruit-class:latest


## Set up estimator and run the traning

In [16]:
estimator = sagemaker.estimator.Estimator(
                       image_uri=image_name,
                       base_job_name=base_job_name,
                       role=role, 
                       instance_count=1, 
                       instance_type=train_instance_type,
                       output_path=output_path,
                       sagemaker_session=sess,
                        #input_mode="Pipe",
                       hyperparameters={
                              'epochs': 20,
                              'batch-size': 130,
                              'lr': 0.01}
                        )

In [ ]:
estimator.fit({'training': train_input_path, 'validation': validation_input_path})

2021-02-20 18:46:55 Starting - Starting the training job...
2021-02-20 18:47:18 Starting - Launching requested ML instancesProfilerReport-1613846814: InProgress
......
2021-02-20 18:48:18 Starting - Preparing the instances for training.........
2021-02-20 18:49:40 Downloading - Downloading input data........................................................................
2021-02-20 19:01:42 Training - Downloading the training image...

### Deploy the trained model weights

In [16]:
predictor=estimator.deploy(initial_instance_count=1,instance_type="ml.m4.xlarge")

-------------------!

In [17]:
from PIL import Image
import numpy as np
import json

In [18]:
image_path="data/training/Banana/176_100.jpg"
img=np.array(Image.open(image_path))
if img is None:
    raise("Image cannot be loaded")
data={"images":img.tolist(),"batch_size":1}
json_data=json.dumps(data)
res=predictor.predict(json_data)
res

In [ ]:
## Delete the endpoint if not needed anymore

In [29]:
predictor.delete_endpoint()